# Tiempo de Prediccion Financiera, en Series tiempo de data con Machine Learning

    Este es un Ejemplo que predice los precios de futuro desde  los movimientos de precio pasados, aqui implentaremos con un ejemplo de tasa EUR/USD, pero podemos convertirlo a precios de stock por la edicion de los simbolos. 

In [ ]:
import datetime
import numpy as np
import pandas as pd
from matplotlib.finance import quotes_historical_yahoo_ochl
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display
%matplotlib inline

print("Setup Complete")

In [ ]:
#time period
d1 = datatime.datatime(2008, 1, 1)
d2 = datatime.datatime(2016, 12, 31)

#Fetching data from yahoo finance
#List of Symbols: http://finance.yahoo.com/webservice/v1/symbols/allcurrencies/quotes
q = quotes_historical_yahoo_ochl('EUR=X', d1, d2, asobject=True)


In [ ]:
#Put data into Pandas DataFrame
df = pd.DataFrame(q)
df.drop(['year', 'month', 'day', 'd'], inplace=True, axis=1)
df[['open', 'high', 'low', 'close', 'aclose']] = df[['open', 'high', 'low', 'close', 'aclose']].applymap(lambda x: 1.0/x)
display(df.head())
df.plot(x='date', y='aclose', figsize=(10,4))

In [ ]:
#Normalize aclose value
#We use this value to train model

df['return'] = df['aclose'] - df['aclose'].shift(1)
return_range = df['return'].max() - df['return'].min()
df['return'] = df['return'] / return_range
df.plot(x='date', y='return', figsize=(10,4))

In [ ]:
#Make label, 1 as rising price, 0 as falling price
df['label'] = df['return'].shift(-1)
df['label'] = df['label'].apply(lambda x: 1 if x>0.0 else 0)

#df.dropna(inplace=True)
df.tail()

In [ ]:
#Make training dataset
n_features = 60  #number of features

train_x = np.array([]).reshape([-1, n_features])
train_y = np.array([]).reshape([-1,1])
for index, row in df.iterrows():
    i = df.index.get_loc(index)
    if i<n_features:
        continue
        
    _x = np.array(df[i-n_features+1:i+1] ['return']).T.reshape([1, -1])
    _y = df.ix[i]['label']
    train_x = np.vstack((train_x, _x))
    train_y = np.vstack((train_y, _y))

train_y =   train_y.reshape([-1])
print(train_x.shape)
print(train_y.shape)
print('%% of Class0 : %f' % (np.count_nonzero(train_y == 0)/float(len(train_y))))
print('%% of Class0 : %f' % (np.count_nonzero(train_y == 1)/float(len(train_y))))

In [ ]:
#Define Model and fit
#Here we use 95% of data for trining, and last 5% for testing

from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=0, learning_rate=0.01, n_estimators=10000)

train_len = int(len(train_x)*0.95)
clf.fit(train_x[:train_len], train_y[:train_len])

accuracy = clf.score(train_x[train_len:], train_y[train_len:])
print('Testing Accuracy: %f' %accuracy)

In [ ]:
#Predict test data
pred = clf.predict(train_x[train_len:])

#Calculate equity..
contracts = 10000.0
commission = 0.0

df_trade = pd.DataFrame(train_x[train_len:,-1], columns=['return'])
df_trade['label'] = train_y[train_len:]
df_trade['pred']  = pred
df_trade['won']   = df_trade['label']== df_trade['pred']
df_trade['return'] = df_trade['return'].shift(-1) * return_range
df_trade.drop(df_trade.index[len(df_trade)-1], inplace=True)

def calc_profit(row):
    if row['won']:
        return abs(row['return'])*contracts - commission
    else:
        return - abs(row['return'])*contracts - commission
    
df_trade['pnl'] = df_trade.apply(lambda row: calc_profit(row), axis=1)
df_trade['equity'] = df_trade['pnl'].cumsum()

display(df_trade.tail())
df_trade.plot(y='equity', figsize=(10,4), title='Backtest with $10000 initial capital')
plt.xlabel('Trades')
plt.labels('Equity (USD)')
for r in df_trade.iterrows():
    if r[1]['won']:
        plt.axvline(x=r[0], linewidth=0.5, alpha=0.8, color='g')
        else:
            plt.axvline(x=r[0], linewidth=0.5, alpha=0.8, color='r')

In [ ]:
#Calculate summary of trades

n_win_trades = float(df_trade[df_trade['pnl']>0.0]['pnl'].count())
n_loss_trades = float(df_trade[df_trade['pnl']<0.0]['pnl'].count()) 

print("Net Profit             :$%.2f" % df_trade.tail(1)['equity'])
print("Number Winning Trades  :%d" % n_win_trades)
print("Number Losing Trades   :%d" % n_loss_trades)
print("Percent Profitable     :%.2f%%" %(100 * n_win_trades/(n_win_trades + n_loss_trades)))
print("Avg Win Trade          :$%.3f" % df_trade[df_trade['pnl']>0.0]['pnl'].mean())
print("Avg Loss Trade         :$%.3f" % df_trade[df_trade['pnl']<0.0]['pnl'].mean())
print("Largest Win Trade      :$%.3f" % df_trade[df_trade['pnl']>0.0]['pnl'].max())
print("Largest Loss Trade     :$%.3f" % df_trade[df_trade['pnl']<0.0]['pnl'].min())
print("Profit Factor          :%.2f" % abs(df_trade[df_trade['pnl']<0.0]['pnl'].sum()/df_trade[df_trade['pnl']<0.0]['pnl'].sum()))

df_trade['pnl'].hist(bins=20)
